In [19]:
from playwright.async_api import async_playwright
import asyncio

In [93]:
async def fetch_with_playwright():
    news_list = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        
        page = await browser.new_page()
        await page.set_extra_http_headers({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
        })
        await page.goto("https://www.prothomalo.com/collection/latest", timeout=60000)
        await page.wait_for_selector("#container", timeout=15000)
        #Wait for a card to appear inside container
        await page.wait_for_selector("#container div.news_with_item", timeout=10000)
        elements = await page.locator("h3.headline-title").all()
        for el in elements:
            link_el  =  el.locator("a.title-link")
            href = await link_el.get_attribute("href")
            title = (await el.inner_text()).strip()
            news_list.append((title, href))
            
        await browser.close()
    return news_list
        #for i in range(articles.count())
     

In [95]:
news_list = await fetch_with_playwright() 
print(news_list)

href https://www.prothomalo.com/bangladesh/capital/jngf17v4j7
href https://www.prothomalo.com/world/india/j7kpxffh1f
href https://www.prothomalo.com/bangladesh/district/wzol6eq3it
href https://www.prothomalo.com/bangladesh/district/4ycfq512uv
href https://www.prothomalo.com/bangladesh/2pamlq750s
href https://www.prothomalo.com/business/industry/ic5yciikxi
href https://www.prothomalo.com/bangladesh/capital/76h5kon04a
href https://www.prothomalo.com/bangladesh/9iobfzpjfn
href https://www.prothomalo.com/sports/cricket/oo0hrwhv6c
href https://www.prothomalo.com/bangladesh/uy2qxp5dp6
href https://www.prothomalo.com/entertainment/entertainment-interview/d8pvdlk80e
href https://www.prothomalo.com/technology/cyberworld/mqrkbta29q
href https://www.prothomalo.com/bangladesh/capital/76h5kon04a
href https://www.prothomalo.com/bangladesh/district/drveov5qrh
href https://www.prothomalo.com/bangladesh/district/tmh0gbqg61
href https://www.prothomalo.com/bangladesh/district/iht5xc8zjz
href https://www.

[('গাজায় ইসরায়েলি আগ্রাসনের প্রতিবাদে আফতাবনগরে বিক্ষোভ',
  'https://www.prothomalo.com/bangladesh/capital/jngf17v4j7'),
 ('রাষ্ট্রপতির সম্মতি, ওয়াক্\u200cফ বিল আইনে পরিণত',
  'https://www.prothomalo.com/world/india/j7kpxffh1f'),
 ('বিদায় অনুষ্ঠানে দাওয়াত না দেওয়ায় ছাত্রদল নেতার হাতে শিক্ষক লাঞ্ছিত',
  'https://www.prothomalo.com/bangladesh/district/wzol6eq3it'),
 ('ডিজিএফআই কর্মকর্তা হত্যার মামলা আরসাপ্রধান আতাউল্লাহকে নেওয়া হলো বান্দরবান আদালতে',
  'https://www.prothomalo.com/bangladesh/district/4ycfq512uv'),
 ('আরও একটি মামলায় সোনালী ব্যাংকের সাবেক এমডি হুমায়ুন কবিরসহ ১১ জনকে তিন বছর কারাদণ্ড',
  'https://www.prothomalo.com/bangladesh/2pamlq750s'),
 ('পাল্টা শুল্ক তিন মাস স্থগিতের অনুরোধ করে ডোনাল্ড ট্রাম্পকে চিঠি দিলেন প্রধান উপদেষ্টা',
  'https://www.prothomalo.com/business/industry/ic5yciikxi'),
 ('ঢাকায় যুক্তরাষ্ট্র দূতাবাসসহ কূটনীতিক এলাকায় নিরাপত্তা জোরদার',
  'https://www.prothomalo.com/bangladesh/capital/76h5kon04a'),
 ('বিক্ষোভের বিষয়ে যুক্তরাষ্ট্রের নাগরিকদের সতর্ক থাকা